In [ ]:
# resources
# https://medium.com/all-things-ai/in-depth-parameter-tuning-for-gradient-boosting-3363992e9bae

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt 
from matplotlib.legend_handler import HandlerLine2D

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc

import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv('../input/titaniccleaningfeatureengineering/trainC.csv')
test = pd.read_csv('../input/titaniccleaningfeatureengineering/testC.csv')
submission = pd.read_csv('../input/titanic/gender_submission.csv')

X = train[train.columns[1:]]
y = train['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
# baseline with default parametes
gbrt = GradientBoostingClassifier()
gbrt.fit(X_train, y_train)
print("accuracy on training set: %f" % gbrt.score(X_train, y_train))
print("accuracy on test set: %f" % gbrt.score(X_test, y_test))

In [ ]:
submission['Survived'] = gbrt.predict(test)
submission.to_csv("submission.csv", index = False)
submission.head()

In [ ]:
# learning_rate
# shrinks the contribution of each tree

learning_rates = [1, 0.5, 0.25, 0.1, 0.05, 0.01]
train_results = []
test_results = []

for eta in learning_rates:
    gbrt = GradientBoostingClassifier(learning_rate=eta)
    gbrt.fit(X_train, y_train)
    train_pred = gbrt.predict(X_train)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    train_results.append(roc_auc)
    y_pred = gbrt.predict(X_test)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_results.append(roc_auc)
    
line1, = plt.plot(learning_rates, train_results, label='Train AUC')
line2, = plt.plot(learning_rates, test_results, label='Test AUC')
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})

plt.ylabel('AUC score')
plt.xlabel('learning rate')
plt.show()

In [ ]:
# N_estimators
# represents the number of trees in the forest
# the higher the number of trees the better to learn the data, but adding a lot of trees slows down the training proces

n_estimators = [1, 2, 4, 8, 16, 32, 64]
train_results = []
test_results = []

for estimator in n_estimators:
    gbrt = GradientBoostingClassifier(n_estimators=estimator)
    gbrt.fit(X_train, y_train)
    train_pred = gbrt.predict(X_train)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    train_results.append(roc_auc)
    y_pred = gbrt.predict(X_test)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_results.append(roc_auc)

line1, = plt.plot(n_estimators, train_results, label='Train AUC')
line2, = plt.plot(n_estimators, test_results, label='Test AUC')
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})

plt.ylabel('AUC score')
plt.xlabel('n_estimators')
plt.show()

In [ ]:
# max_depth
# how deep the built tree can be (the deeper the tree, the more information it captures)

max_depths = np.linspace(1, 32, 32, endpoint=True)
train_results = []
test_results = []

for max_depth in max_depths:
    gbrt = GradientBoostingClassifier(max_depth=max_depth)
    gbrt.fit(X_train, y_train)
    train_pred = gbrt.predict(X_train)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    train_results.append(roc_auc)
    y_pred = gbrt.predict(X_test)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_results.append(roc_auc)

line1, = plt.plot(max_depths, train_results, label='Train AUC')
line2, = plt.plot(max_depths, test_results, label='Test AUC')
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})

plt.ylabel('AUC score')
plt.xlabel('n_estimators')
plt.show()

In [ ]:
# min_samples_split
# represents the minimum number of samples required to split an internal node
# increase this parameter, the tree becomes more constrained as it has to consider more samples at each node

min_samples_splits = np.linspace(0.1, 1.0, 10, endpoint=True)
train_results = []
test_results = []

for min_samples_split in min_samples_splits:
    gbrt = GradientBoostingClassifier(min_samples_split=min_samples_split)
    gbrt.fit(X_train, y_train)
    train_pred = gbrt.predict(X_train)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    train_results.append(roc_auc)
    y_pred = gbrt.predict(X_test)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_results.append(roc_auc)

line1, = plt.plot(min_samples_splits, train_results, label='Train AUC')
line2, = plt.plot(min_samples_splits, test_results, label='Test AUC')
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})

plt.ylabel('AUC score')
plt.xlabel('min samples splits')
plt.show()

In [ ]:
# min_samples_leaf
# the minimum number of samples required to be at a leaf node

min_samples_leafs = np.linspace(0.1, 0.5, 5, endpoint=True)
train_results = []
test_results = []

for min_samples_leaf in min_samples_leafs:
    gbrt = GradientBoostingClassifier(min_samples_leaf=min_samples_leaf)
    gbrt.fit(X_train, y_train)
    train_pred = gbrt.predict(X_train)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    train_results.append(roc_auc)
    y_pred = gbrt.predict(X_test)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_results.append(roc_auc)
    
line1, = plt.plot(min_samples_leafs, train_results, label='Train AUC')
line2, = plt.plot(min_samples_leafs, test_results, label='Test AUC')
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})

plt.ylabel('AUC score')
plt.xlabel('min samples leaf')
plt.show()

In [ ]:
# max_features
# represents the number of features to consider when looking for the best split

max_features = list(range(1,train.shape[1]))
train_results = []
test_results = []

for max_feature in max_features:
    gbrt = GradientBoostingClassifier(max_features=max_feature)
    gbrt.fit(X_train, y_train)
    train_pred = gbrt.predict(X_train)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    train_results.append(roc_auc)
    y_pred = gbrt.predict(X_test)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_results.append(roc_auc)

line1, = plt.plot(max_features, train_results, label='Train AUC')
line2, = plt.plot(max_features, test_results, label='Test AUC')
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})

plt.ylabel('AUC score')
plt.xlabel('max features')
plt.show()

In [ ]:
# optimized parameters
#gbrt_opt = GradientBoostingClassifier( min_samples_split=0.7)
#gbrt_opt.fit(X_train, y_train)
#print("accuracy on training set: %f" % gbrt_opt.score(X_train, y_train))
#print("accuracy on test set: %f" % gbrt_opt.score(X_test, y_test))

In [ ]:
learning_rates = [1, 0.5, 0.25, 0.1, 0.05, 0.01]
n_estimators = [1, 2, 4, 8, 16, 32, 64]
max_depths = np.linspace(1, 32, 32, endpoint=True)
min_samples_splits = np.linspace(0.1, 1.0, 10, endpoint=True)
min_samples_leafs = np.linspace(0.1, 0.5, 5, endpoint=True)
max_features = list(range(1,train.shape[1]))

param = { 'learning_rate' : learning_rates, 
         'n_estimators' : n_estimators, 
         'max_depth' : max_depths, 
         'min_samples_split' : min_samples_splits, 
         'min_samples_leaf' : min_samples_leafs,
         'max_features' : max_features,
        }
cv = StratifiedShuffleSplit(n_splits=10, test_size=.30, random_state=15)
grid = GridSearchCV(GradientBoostingClassifier(), param, cv=cv, verbose = False, n_jobs=-1)
grid.fit(X,y)

In [ ]:
print (grid.best_score_)
print (grid.best_params_)
print(grid.best_estimator_)

In [ ]:
submission['Survived'] = grid.predict(test)
submission.to_csv("submission_GridSearchCV.csv", index = False)
submission.head()